In [11]:
import random
import json
import pickle
import numpy as np
import keras
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential

from keras.layers.core import Dense, Activation, Dropout 
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())


words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
            bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

sgd = SGD(lr = 0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size = 5, verbose=1)
model.save("chatbot.h5", hist)
print("Done")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shurk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-11-f9a7ebb4ec7f>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
6/6 [==============================] - 0s 831us/step - loss: 1.8186 - accuracy: 0.1000
Epoch 2/200
6/6 [==============================] - 0s 665us/step - loss: 1.7474 - accuracy: 0.1667
Epoch 3/200
6/6 [==============================] - 0s 997us/step - loss: 1.7044 - accuracy: 0.3333
Epoch 4/200
6/6 [==============================] - 0s 997us/step - loss: 1.7004 - accuracy: 0.3000
Epoch 5/200
6/6 [==============================] - 0s 997us/step - loss: 1.6205 - accuracy: 0.4000
Epoch 6/200
6/6 [==============================] - 0s 665us/step - loss: 1.6710 - accuracy: 0.3333
Epoch 7/200
6/6 [==============================] - 0s 832us/step - loss: 1.5113 - accuracy: 0.3667
Epoch 8/200
6/6 [==============================] - 0s 958us/step - loss: 1.6654 - accuracy: 0.4000
Epoch 9/200
6/6 [==============================] - 0s 665us/step - loss: 1.4046 - accuracy: 0.5667
Epoch 10/200
6/6 [==============================] - 0s 997us/step - loss: 1.4018 - accuracy: 0.5333
Epoch 11/